<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

In [2]:
spark

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

meteoDataFrame  = spark.read.format('csv')\
    .option('sep',';')\
    .option('header','true')\
    .option('nullValue','mq')\
    .option('inferSchema', 'true')\
    .load('./donnees/meteo/2024')\
    .cache()

schema = StructType([
        StructField('Id'           , StringType() , True),
        StructField('ville'        , StringType() , True),
        StructField('latitude'     , FloatType() , True),
        StructField('longitude'    , FloatType() , True),
        StructField('altitude'     , IntegerType() , True)])

villes  = spark.read.format('csv')   \
      .option('sep',';')                \
      .option('mergeSchema', 'true')    \
      .option('header','true')          \
      .schema(schema)                   \
      .load('./donnees/postesSynop.csv')  \
      .cache()

@udf("string")
def formatVille(ville):
    if ville in ['CLERMONT-FD','MONT-DE-MARSAN',
                                   'ST-PIERRE','ST-BARTHELEMY METEO'] :
        return ville.title()
    else :
        if ville.find('-') != -1 :
            return ville[0:ville.find('-')].title()
        else:
            return ville.title()

villesT  = villes.select(
                col('Id').alias('id'),
                formatVille('ville').alias('ville'),
               'latitude',
               'longitude',
               'altitude')


meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 coalesce( col('rr3'),
                           col('rr24')/8,
                           col('rr12')/4,
                           col('rr6')/2,
                           col('rr1')*3  ) )\
             .toDF('id','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression','precipitations')\
             .cache()

meteo.select('annee','mois','jour','temperature','humidite',
             'visibilite','pression').toPandas().head(5)

25/09/16 14:51:15 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/09/16 14:51:15 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/09/16 14:51:22 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/09/16 14:51:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/09/16 14:51:22 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.exe

,annee,mois,jour,temperature,humidite,visibilite,pression
0,2024,5,1,13.1,0.94,14.20,100.13
1,2024,5,1,14.5,0.85,16.03,100.56
2,2024,5,1,8.8,0.88,NaN,100.74
3,2024,5,1,9.2,0.91,11.05,100.14
4,2024,5,1,12.6,0.97,14.33,99.08


In [4]:
meteo.write\
       .mode('overwrite')\
       .format('parquet')\
       .partitionBy('annee')\
       .option('path', 'donnees/meteoFrance')\
       .save()

In [5]:
!ls -al donnees

total 20
drwxr-xr-x  4 spark hadoop 4096 sept. 16 14:52 .
drwxr-x--- 15 spark hadoop 4096 sept. 16 14:39 ..
drwxr-xr-x 32 spark hadoop 4096 sept. 16 14:33 meteo
drwxr-xr-x  3 spark hadoop 4096 sept. 16 14:52 meteoFrance
-rw-r--r--  1 spark hadoop 2618 sept. 16 14:35 postesSynop.csv


In [8]:
spark.sql("select * from parquet."+
          "`./donnees/meteoFrance` "+
          "where annee = 2024 and mois = 1").toPandas().head(5)

25/09/16 14:53:25 WARN ObjectStore: Failed to get database parquet, returning NoSuchObjectException


,id,mois,jour,mois_jour,temperature,humidite,visibilite,pression,precipitations,annee
0,7005,1,1,0101,7.4,0.81,15.46,99.32,1.0,2024
1,7015,1,1,0101,7.5,0.85,19.85,99.45,1.2,2024
2,7020,1,1,0101,10.9,0.71,NaN,100.16,0.0,2024
3,7027,1,1,0101,7.3,0.77,24.10,99.71,0.6,2024
4,7037,1,1,0101,6.5,0.81,41.60,98.61,0.0,2024


In [9]:
meteoFance = spark.read.format('parquet').load('./donnees/meteoFrance')

In [10]:
spark.sql('DROP DATABASE IF EXISTS cours CASCADE').toPandas().head()

25/09/16 14:53:37 WARN ObjectStore: Failed to get database cours, returning NoSuchObjectException


""


In [11]:
spark.sql('CREATE DATABASE cours').toPandas().head()

25/09/16 14:53:41 WARN ObjectStore: Failed to get database cours, returning NoSuchObjectException
25/09/16 14:53:41 WARN ObjectStore: Failed to get database cours, returning NoSuchObjectException
25/09/16 14:53:41 WARN ObjectStore: Failed to get database cours, returning NoSuchObjectException


""


In [12]:
spark.sql('show databases').toPandas()

,namespace
0,cours
1,default


In [13]:
meteo=spark.sql("""
                SELECT * 
                FROM parquet.`./donnees/meteoFrance`""") 

25/09/16 14:53:52 WARN ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [14]:
meteo.toPandas().head()

,id,mois,jour,mois_jour,temperature,humidite,visibilite,pression,precipitations,annee
0,7005,5,1,0501,13.1,0.94,14.20,100.13,0.2,2024
1,7015,5,1,0501,14.5,0.85,16.03,100.56,-0.1,2024
2,7020,5,1,0501,8.8,0.88,NaN,100.74,0.0,2024
3,7027,5,1,0501,9.2,0.91,11.05,100.14,0.0,2024
4,7037,5,1,0501,12.6,0.97,14.33,99.08,0.0,2024


In [15]:
spark.sql('USE cours').toPandas()
meteo.write.saveAsTable(name="meteo", mode="overwrite")

25/09/16 14:54:00 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/09/16 14:54:00 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist


In [16]:
spark.sql('show tables').toPandas()

,namespace,tableName,isTemporary
0,cours,meteo,False


In [17]:
spark.sql("""SELECT *
            FROM meteo
            WHERE ANNEE = 2024""").toPandas().head(10)

,id,mois,jour,mois_jour,temperature,humidite,visibilite,pression,precipitations,annee
0,7005,5,1,0501,13.1,0.94,14.20,100.13,0.2,2024
1,7015,5,1,0501,14.5,0.85,16.03,100.56,-0.1,2024
2,7020,5,1,0501,8.8,0.88,NaN,100.74,0.0,2024
3,7027,5,1,0501,9.2,0.91,11.05,100.14,0.0,2024
4,7037,5,1,0501,12.6,0.97,14.33,99.08,0.0,2024
5,7072,5,1,0501,12.8,0.93,18.39,100.01,1.2,2024
6,7110,5,1,0501,7.8,0.94,33.68,99.62,-0.1,2024
7,7117,5,1,0501,8.5,0.92,NaN,100.11,-0.1,2024
8,7130,5,1,0501,10.2,0.90,19.73,100.50,0.2,2024
9,7139,5,1,0501,10.1,0.95,20.00,99.18,0.0,2024


In [18]:
spark.sql("""SELECT annee, 
                    avg(temperature) temperature, 
                    avg(humidite) humidite, 
                    avg(visibilite) visibilite, 
                    avg(pression) pression
            FROM meteo
            GROUP BY ANNEE""").toPandas().head(20)

,annee,temperature,humidite,visibilite,pression
0,2024,15.698337,0.78093,25.625652,99.954934


In [19]:
spark.sql('USE cours').toPandas()

""


In [20]:
spark.sql('show tables').toPandas()

,namespace,tableName,isTemporary
0,cours,meteo,False
